In [4]:
import pandas as pd
import re
from pathlib import Path

In [5]:
path = Path("../1.raw/ds_salaries.csv")
df1 = pd.read_csv(path)

path = Path("../1.raw/global_tech_salary.txt")
df2 = pd.read_csv(path)

path = Path("../1.raw/salary_data_cleaned.csv")
df3 = pd.read_csv(path)

In [6]:
FINAL_COLS = [
    "source_dataset",
    "work_year",
    "role_raw",
    "role_label",
    "seniority_raw",
    "seniority",
    "country",
    "state_or_region",
    "remote_ratio",
    "company_size",
    "employment_type",
    "salary_in_usd",
]

In [7]:
import numpy as np

# -----------------------------
# Seniority a partir de experience_level (EN/MI/SE/EX)
# -----------------------------
def map_experience_level_to_seniority(exp_level: str) -> str:
    if pd.isna(exp_level):
        return np.nan
    exp = str(exp_level).strip().upper()
    if exp == "EN":
        return "Junior"
    elif exp == "MI":
        return "Mid"
    elif exp in ("SE", "EX"):
        return "Senior"
    else:
        return np.nan


# -----------------------------
# Seniority a partir de job title
# (para Glassdoor: Senior, Lead, Principal, Director, etc.)
# -----------------------------
SENIOR_PATTERNS = [
    r"\bsenior\b", r"\bsr\b", r"\bprincipal\b", r"\blead\b",
    r"\bmanager\b", r"\bdirector\b", r"\bhead\b"
]
JUNIOR_PATTERNS = [
    r"\bjunior\b", r"\bjr\b", r"\bentry\b", r"\bentry-level\b"
]

def map_title_to_seniority(title: str) -> str:
    if pd.isna(title):
        return np.nan
    t = str(title).lower()
    # senior primero
    for pat in SENIOR_PATTERNS:
        if re.search(pat, t):
            return "Senior"
    # luego junior
    for pat in JUNIOR_PATTERNS:
        if re.search(pat, t):
            return "Junior"
    # si no hay señal → asumimos Mid
    return "Mid"


# -----------------------------
# Normalizar rol a tu taxonomía base (data roles)
# -----------------------------
def normalize_role(job_title: str) -> str:
    """
    Mapea job_title bruto a un role_label normalizado.
    Ajusta o amplía reglas cuando veas casos raros.
    """
    if pd.isna(job_title):
        return "other"

    t = str(job_title).lower()

    # Orden importa: de más específico a más general
    if "machine learning engineer" in t or "ml engineer" in t:
        return "machine_learning_engineer"
    if "data engineer" in t or "analytics engineer" in t:
        return "data_engineer"
    if "data scientist" in t:
        return "data_scientist"
    if "research scientist" in t or "applied scientist" in t:
        return "research_scientist"
    if "data analyst" in t or "business analyst" in t:
        return "data_analyst"
    if "bi engineer" in t or "business intelligence engineer" in t:
        return "bi_engineer"
    if "data architect" in t:
        return "data_architect"
    if "manager" in t:
        return "data_manager"
    if "director" in t or "head of" in t:
        return "data_director"

    # fallback genérico
    if "data" in t:
        return "other_data_role"
    return "other"


# -----------------------------
# Filtro de outliers por cuantiles
# -----------------------------
def clip_salary_outliers(df: pd.DataFrame, col: str = "salary_in_usd",
                         lower_q: float = 0.01, upper_q: float = 0.99) -> pd.DataFrame:
    """
    Recorta filas con salarios fuera de [q_lower, q_upper].
    Devuelve un df filtrado (no modifica in place).
    """
    df = df.copy()
    q_low, q_high = df[col].quantile([lower_q, upper_q])
    return df[(df[col] >= q_low) & (df[col] <= q_high)]


In [8]:
def normalize_ds_style_df(df: pd.DataFrame, source_name: str) -> pd.DataFrame:
    """
    Normaliza un dataframe tipo ds_salaries/global_tech_salary
    al esquema FINAL_COLS.
    """
    # Copia para no tocar el original
    df = df.copy()

    # Eliminar duplicados exactos
    df = df.drop_duplicates()

    # Asegurarnos de que existen columnas básicas
    required_cols = [
        "work_year", "job_title", "experience_level",
        "salary_in_usd", "company_location"
    ]
    for c in required_cols:
        if c not in df.columns:
            raise ValueError(f"Columna requerida '{c}' no encontrada en df ({source_name})")

    # Filtrar outliers de salario
    df = df[df["salary_in_usd"].notna()]
    df = clip_salary_outliers(df, col="salary_in_usd", lower_q=0.01, upper_q=0.99)

    # Construimos el df normalizado
    out = pd.DataFrame(index=df.index)

    out["source_dataset"] = source_name
    out["work_year"] = df["work_year"].astype("Int64")

    out["role_raw"] = df["job_title"].astype(str)
    out["role_label"] = df["job_title"].astype(str).apply(normalize_role)

    out["seniority_raw"] = df["experience_level"].astype(str)
    out["seniority"] = df["experience_level"].astype(str).apply(map_experience_level_to_seniority)

    out["country"] = df["company_location"].astype(str)

    # Estos no los tenemos a nivel de estado/region en estos datasets
    out["state_or_region"] = pd.NA

    # remote_ratio, company_size, employment_type pueden faltar en algún dataset
    if "remote_ratio" in df.columns:
        out["remote_ratio"] = df["remote_ratio"]
    else:
        out["remote_ratio"] = pd.NA

    if "company_size" in df.columns:
        out["company_size"] = df["company_size"]
    else:
        out["company_size"] = pd.NA

    if "employment_type" in df.columns:
        out["employment_type"] = df["employment_type"]
    else:
        out["employment_type"] = pd.NA

    out["salary_in_usd"] = df["salary_in_usd"].astype(float)

    # Nos aseguramos del orden de columnas:
    out = out[FINAL_COLS]

    return out


In [9]:
ds_norm = normalize_ds_style_df(df1, source_name="ds_salaries")
global_tech_norm = normalize_ds_style_df(df2, source_name="global_tech_salary")

print(ds_norm.shape, global_tech_norm.shape)
ds_norm.head()


(593, 12) (3778, 12)


,source_dataset,work_year,role_raw,role_label,seniority_raw,seniority,country,state_or_region,remote_ratio,company_size,employment_type,salary_in_usd
0,ds_salaries,2020,Data Scientist,data_scientist,MI,Mid,DE,<NA>,0,L,FT,79833.0
1,ds_salaries,2020,Machine Learning Scientist,other,SE,Senior,JP,<NA>,0,S,FT,260000.0
2,ds_salaries,2020,Big Data Engineer,data_engineer,SE,Senior,GB,<NA>,50,M,FT,109024.0
3,ds_salaries,2020,Product Data Analyst,data_analyst,MI,Mid,HN,<NA>,0,S,FT,20000.0
4,ds_salaries,2020,Machine Learning Engineer,machine_learning_engineer,SE,Senior,US,<NA>,50,L,FT,150000.0


In [10]:
def normalize_glassdoor_df(df: pd.DataFrame, source_name: str = "glassdoor_ds") -> pd.DataFrame:
    """
    Normaliza salary_data_cleaned.csv al esquema FINAL_COLS.

    Suposiciones:
    - 'avg_salary' está en miles de USD/año (ej. 100 → 100k).
    - 'hourly' indica salarios por hora (los filtramos).
    - Todo es USA (country = 'US').
    """
    df = df.copy()

    # Comprobaciones mínimas de columnas
    required_cols = ["Job Title", "avg_salary"]
    for c in required_cols:
        if c not in df.columns:
            raise ValueError(f"Columna requerida '{c}' no encontrada en df Glassdoor: {c}")

    # Filtrar salarios por hora (hourly == 1)
    if "hourly" in df.columns:
        df = df[df["hourly"] == 0].copy()

    # Construimos salary_in_usd a partir de avg_salary (en miles)
    df = df[df["avg_salary"].notna()]
    df["salary_in_usd"] = df["avg_salary"].astype(float) * 1000.0

    # Filtrar outliers
    df = clip_salary_outliers(df, col="salary_in_usd", lower_q=0.01, upper_q=0.99)

    out = pd.DataFrame(index=df.index)

    out["source_dataset"] = source_name

    # No tenemos año explícito → NA (si quieres, puedes meter un año fijo)
    out["work_year"] = pd.NA

    out["role_raw"] = df["Job Title"].astype(str)
    out["role_label"] = df["Job Title"].astype(str).apply(normalize_role)

    # Si ya existiera una columna 'seniority' en el csv, podemos usarla como raw
    if "seniority" in df.columns:
        out["seniority_raw"] = df["seniority"].astype(str)
        # Y mapear a nuestro esquema si quieres, pero aquí usamos title para mantener lógica homogénea
        out["seniority"] = df["Job Title"].astype(str).apply(map_title_to_seniority)
    else:
        out["seniority_raw"] = df["Job Title"].astype(str)
        out["seniority"] = df["Job Title"].astype(str).apply(map_title_to_seniority)

    # Localización: asumimos USA
    out["country"] = "US"

    # job_state → state_or_region si existe
    if "job_state" in df.columns:
        out["state_or_region"] = df["job_state"].astype(str)
    else:
        out["state_or_region"] = pd.NA

    # No tenemos remote_ratio en este dataset
    out["remote_ratio"] = pd.NA

    # Tamaño empresa: si existe 'Size'
    if "Size" in df.columns:
        out["company_size"] = df["Size"].astype(str)
    else:
        out["company_size"] = pd.NA

    # employment_type: suponemos FT (full-time) porque son ofertas estándar
    out["employment_type"] = "FT"

    out["salary_in_usd"] = df["salary_in_usd"].astype(float)

    out = out[FINAL_COLS]

    return out


In [11]:
glassdoor_norm = normalize_glassdoor_df(df3, source_name="glassdoor_ds")

print(glassdoor_norm.shape)
glassdoor_norm.head()


(702, 12)


,source_dataset,work_year,role_raw,role_label,seniority_raw,seniority,country,state_or_region,remote_ratio,company_size,employment_type,salary_in_usd
0,glassdoor_ds,<NA>,Data Scientist,data_scientist,Data Scientist,Mid,US,NM,<NA>,501 to 1000 employees,FT,72000.0
1,glassdoor_ds,<NA>,Healthcare Data Scientist,data_scientist,Healthcare Data Scientist,Mid,US,MD,<NA>,10000+ employees,FT,87500.0
2,glassdoor_ds,<NA>,Data Scientist,data_scientist,Data Scientist,Mid,US,FL,<NA>,501 to 1000 employees,FT,85000.0
3,glassdoor_ds,<NA>,Data Scientist,data_scientist,Data Scientist,Mid,US,WA,<NA>,1001 to 5000 employees,FT,76500.0
4,glassdoor_ds,<NA>,Data Scientist,data_scientist,Data Scientist,Mid,US,NY,<NA>,51 to 200 employees,FT,114500.0


In [12]:
# -----------------------------
# Concatenar los tres datasets normalizados
# -----------------------------
all_salaries = pd.concat(
    [ds_norm, global_tech_norm, glassdoor_norm],
    ignore_index=True
)

print("Shape antes de quitar duplicados:", all_salaries.shape)

# Quitar duplicados aproximados (por perfil + salario)
dup_subset = ["source_dataset", "work_year", "role_label",
              "seniority", "country", "state_or_region", "salary_in_usd"]

all_salaries = all_salaries.drop_duplicates(subset=dup_subset).reset_index(drop=True)

print("Shape después de quitar duplicados:", all_salaries.shape)

# Un pequeño check de distribución
print("\nDistribución de seniority:")
print(all_salaries["seniority"].value_counts(dropna=False))

print("\nDistribución de roles normalizados:")
print(all_salaries["role_label"].value_counts().head(15))

print("\nEjemplo de filas:")
all_salaries.head()


Shape antes de quitar duplicados: (5073, 12)
Shape después de quitar duplicados: (4309, 12)

Distribución de seniority:
Senior    2538
Mid       1393
Junior     378
Name: seniority, dtype: int64

Distribución de roles normalizados:
data_scientist               988
data_engineer                972
data_analyst                 658
other                        469
machine_learning_engineer    432
other_data_role              257
research_scientist           183
data_manager                 132
data_architect               108
data_director                 60
bi_engineer                   50
Name: role_label, dtype: int64

Ejemplo de filas:


,source_dataset,work_year,role_raw,role_label,seniority_raw,seniority,country,state_or_region,remote_ratio,company_size,employment_type,salary_in_usd
0,ds_salaries,2020,Data Scientist,data_scientist,MI,Mid,DE,<NA>,0,L,FT,79833.0
1,ds_salaries,2020,Machine Learning Scientist,other,SE,Senior,JP,<NA>,0,S,FT,260000.0
2,ds_salaries,2020,Big Data Engineer,data_engineer,SE,Senior,GB,<NA>,50,M,FT,109024.0
3,ds_salaries,2020,Product Data Analyst,data_analyst,MI,Mid,HN,<NA>,0,S,FT,20000.0
4,ds_salaries,2020,Machine Learning Engineer,machine_learning_engineer,SE,Senior,US,<NA>,50,L,FT,150000.0


In [13]:
OUTPUT_DIR = Path("../3.processed")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_PATH = OUTPUT_DIR / "salaries_unified.csv"

all_salaries.to_csv(OUTPUT_PATH, index=False)
print(f"\n✅ Dataset unificado guardado en: {OUTPUT_PATH}")



✅ Dataset unificado guardado en: ..\3.processed\salaries_unified.csv
